In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import from_json, col, expr
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StringType, TimestampType
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.5,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5 pyspark-shell'

In [2]:
spark =  SparkSession.builder \
    .appName("BTCUSDT Bonus") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.5") \
    .getOrCreate()

25/05/06 17:07:40 WARN Utils: Your hostname, lnvi-legion resolves to a loopback address: 127.0.1.1; using 192.168.100.210 instead (on interface wlp4s0)
25/05/06 17:07:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/lnvi/Code/real-time-binance-pipeline/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lnvi/.ivy2/cache
The jars for the packages stored in: /home/lnvi/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-843849c3-39c8-4c08-9870-ad0d3660b1a7;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.5 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.5 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.

In [3]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "btc-price") \
    .load()

In [ ]:
schema = StructType() \
    .add("symbol", StringType()) \
    .add("price", StringType()) \
    .add("event-time", TimestampType())

parsed_df = df.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), schema).alias("data")) \
    .select("data.symbol", col("data.price").cast("double"), "data.event-time")

query = parsed_df.writeStream \
    .format("console") \
    .option("truncate", "false") \
    .start()
query.awaitTermination()

25/05/06 17:07:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-57ec417e-d734-4246-86a7-e7b7a225a7f0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/06 17:07:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/06 17:07:51 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+--------------------------+----------+
|symbol|CAST(data.price AS DOUBLE)|event-time|
+------+--------------------------+----------+
+------+--------------------------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.24                  |2025-05-06 17:07:58.202184|
+-------+--------------------------+--------------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.23                  |2025-05-06 17:07:58.438437|
|BTCUSDT|94182.23                  |2025-05-06 17:07:58.659163|
|BTCUSDT|94182.23                  |2025-05-06 17:07:58.882098|
|BTCUSDT|94182.24                  |2025-05-06 17:07:59.131548|
|BTCUSDT|94182.24                  |2

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/lnvi/Code/real-time-binance-pipeline/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lnvi/Code/real-time-binance-pipeline/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


-------------------------------------------
Batch: 23
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.23                  |2025-05-06 17:08:05.453485|
+-------+--------------------------+--------------------------+

-------------------------------------------
Batch: 24
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.23                  |2025-05-06 17:08:05.679584|
+-------+--------------------------+--------------------------+

-------------------------------------------
Batch: 25
-------------------------------------------
+-------+--------------------------+--------------------------+


KeyboardInterrupt: 

-------------------------------------------
Batch: 26
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.23                  |2025-05-06 17:08:06.131387|
+-------+--------------------------+--------------------------+

-------------------------------------------
Batch: 27
-------------------------------------------
+-------+--------------------------+--------------------------+
|symbol |CAST(data.price AS DOUBLE)|event-time                |
+-------+--------------------------+--------------------------+
|BTCUSDT|94182.23                  |2025-05-06 17:08:06.371181|
+-------+--------------------------+--------------------------+

-------------------------------------------
Batch: 28
-------------------------------------------
+-------+--------------------------+--------------------------+


In [ ]:
watermarked_df = df.withWatermark("event_time", "10 minutes")

In [ ]:
join_df = watermarked_df.alias("a").join(
    watermarked_df.alias("b"),
    (col("a.event_time") < col("b.event_time")) &
    (col("a.event_time")+ expr("INTERVAL 20 seconds") >= col("b.event_time") ),
    "leftOuter"
)

In [ ]:
window = Window.partitionBy("a.event_time").orderBy("event_time")

higher_df = join_df.select("a.price < b.price") \
    .withColumn("rank", expr("row_number() over (partition by a.event_time order by event_time)")) \
    .filter(col("rank") == 1) \
    .select("a.event_time", "a.price", "b.price") \
    .withColumnRenamed("a.price", "price_a") \
    .withColumnRenamed("b.price", "price_b")


In [ ]:
query = parsed_df.writeStream \
    .format("console") \
    .option("truncate", "false") \
    .start()
query.awaitTermination()

In [5]:
spark.stop()

25/05/06 17:07:05 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 241, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/05/06 17:07:05 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 241, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/05/06 17:07:05 ERROR MicroBatchExecution: Query [id = 8f29fd75-60e5-4d33-ad1b-52df64014242, runId = 9aedfe31-4959-43d1-a158-9d5172160293] terminated with error
org.apache.spark.SparkException: Job 241 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGScheduler